<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Attention_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Introduction



Train a model to estimate blood pressure: systolic and diastolic on 60s input PPG data

#2. Setup Environment

In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, BatchNormalization, Input, Add, Activation,\
MaxPooling1D,Dropout,Flatten,TimeDistributed,Bidirectional,Dense,LSTM, ZeroPadding1D, \
AveragePooling1D,GlobalMaxPooling1D, Concatenate, Permute, Dot, Multiply, RepeatVector,\
Lambda, Average
from tensorflow.keras.initializers import glorot_uniform
import tensorflow_datasets as tfds
import multiprocessing
from datetime import datetime
import sklearn.metrics
import itertools
import io
import pickle
print(tf.__version__)

2.2.0


#3. Data Pipeline

In [0]:
#load the data filename
train_data_dir = r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\train\data_3*"
train_data_fn = glob.glob(train_data_dir)
train_label_dir = r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\train\label_3*"
train_label_fn = glob.glob(train_label_dir)

In [0]:
#run assert to make sure the data and label are in the same order
for i in range(len(train_label_fn)):
  assert(train_data_fn[i][-1] == train_label_fn[i][-1])

In [0]:
val_data_dir = r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\validation\data_3*"
val_data_fn = glob.glob(val_data_dir)
val_label_dir = r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\validation\label_3*"
val_label_fn = glob.glob(val_label_dir)

In [0]:
for i in range(len(val_label_fn)):
  assert(val_data_fn[i][-1] == val_label_fn[i][-1])

In [0]:
#use generator to input data, since the data size(>160GB) is larger than memory size (64GB)
def train_data_generator():
  for i in range(len(train_data_fn)):
    data = pickle.load(open(train_data_fn[i],'rb'))
    yield data

In [0]:
def train_label_generator():
  for i in range(len(train_label_fn)):
    label = pickle.load(open(train_label_fn[i],'rb'))
    yield label

In [0]:
def val_data_generator():
  for i in range(len(val_data_fn)):
    data = pickle.load(open(val_data_fn[i],'rb'))
    yield data

In [0]:
def val_label_generator():
  for i in range(len(val_label_fn)):
    label = pickle.load(open(val_label_fn[i],'rb'))
    yield label

In [49]:
#calculate number of elements in training for later use in shuffle and model.fit
number_of_element = 0
for i in range(len(train_label_fn)):
  label = pickle.load(open(train_label_fn[i],'rb'))
  number_of_element += len(label)
print("There are in total", number_of_element, "in training dataset")

There are in total 19490 in training dataset


In [50]:
#calculate number of elements in validation
number_of_val_element = 0
for i in range(len(val_label_fn)):
  label = pickle.load(open(val_label_fn[i],'rb'))
  number_of_val_element += len(label)
print("There are in total", number_of_val_element, "in validation dataset")

There are in total 2183 in validation dataset


In [0]:
n_a=32 #Number of pre-LSTM states
n_s=16 #Number of post-LSTM states
t_s0 = np.zeros((number_of_element, n_s))
t_c0 = np.zeros((number_of_element, n_s))

In [0]:
#input the data by using generator and use flat_map to removing the 
#first dimension (number of elements) and flat all data
train_data = tf.data.Dataset.from_generator(train_data_generator,(tf.float32),output_shapes=[None,10,750,1])
train_label = tf.data.Dataset.from_generator(train_label_generator,(tf.float32),output_shapes=[None,2])
train_ds = train_data.flat_map(lambda x: train_data.from_tensor_slices(x))
train_lb = train_label.flat_map(lambda x: train_label.from_tensor_slices(x))
train_s0 = tf.data.Dataset.from_tensor_slices(t_s0)
train_c0 = tf.data.Dataset.from_tensor_slices(t_c0)
train = tf.data.Dataset.zip(((train_ds,train_s0,train_c0),train_lb))

In [0]:
v_s0 = np.zeros((number_of_val_element, n_s))
v_c0 = np.zeros((number_of_val_element, n_s))

In [0]:
#do the same to validation
val_data = tf.data.Dataset.from_generator(val_data_generator,(tf.float32),output_shapes=[None,10,750,1])
val_label = tf.data.Dataset.from_generator(val_label_generator,(tf.float32),output_shapes=[None,2])
val_ds = val_data.flat_map(lambda x: val_data.from_tensor_slices(x))
val_lb = val_label.flat_map(lambda x: val_label.from_tensor_slices(x))
val_s0 = tf.data.Dataset.from_tensor_slices(v_s0)
val_c0 = tf.data.Dataset.from_tensor_slices(v_c0)
validation = tf.data.Dataset.zip(((val_ds,val_s0, val_c0),val_lb))

In [0]:
batch_size = 64
train_dataset = train.cache()
train_dataset = train_dataset.shuffle(number_of_element//10).repeat().batch(batch_size,drop_remainder=True)
train_dataset = train_dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
val_dataset = validation.repeat().batch(batch_size, drop_remainder=True)

In [91]:
train_dataset

<PrefetchDataset shapes: (((64, 10, 750, 1), (64, 16), (64, 16)), (64, 2)), types: ((tf.float32, tf.float64, tf.float64), tf.float32)>

In [92]:
val_dataset

<BatchDataset shapes: (((64, 10, 750, 1), (64, 16), (64, 16)), (64, 2)), types: ((tf.float32, tf.float64, tf.float64), tf.float32)>

#4. Train Model

##4.1 Define model

In [0]:
def identity_block_18(X, f, filters, stage, block):

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv1D(filters = F1, kernel_size = f, strides = 1, padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 2, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path 
    X = Conv1D(filters = F2, kernel_size = f, strides = 1, padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 2, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [0]:
def ResNet18(input_shape=(750, 1), classes=1):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding1D(3)(X_input)

    # Stage 1
    X = Conv1D(64, 7, strides=2, name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=2, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(3, strides=2)(X)

    # Stage 2
    X = identity_block_18(X, 3, [64, 64], stage=2, block='a')
    X = identity_block_18(X, 3, [64, 64], stage=2, block='b')


    # Stage 3 (2 lines)
    X = convolutional_block_18(X, f = 3, filters = [128, 128], stage = 3, block='a', s = 2)
    X = identity_block_18(X, 3, [128, 128], stage=3, block='b')


    # Stage 4 (2 lines)
    X = convolutional_block_18(X, f = 3, filters = [256, 256], stage = 4, block='a', s = 2)
    X = identity_block_18(X, 3, [256, 256], stage=4, block='b')

    # Stage 5 (2 lines)
    X = convolutional_block_18(X, f = 3, filters = [512, 512], stage = 5, block='a', s = 2)
    X = identity_block_18(X, 3, [512, 512], stage=5, block='b')


    # AVGPOOL (1 line).
    X = AveragePooling1D(2, name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    
    
    #X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    #Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet18')

    return model

In [0]:
def convolutional_block_18(X, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv1D(filters = F1, kernel_size = f, strides = s, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 2, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv1D(filters = F2, kernel_size = f, strides = 1, padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 2, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv1D(filters = F1, kernel_size = f, strides = s, padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 2, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [0]:
def one_step_attention(a, s_prev): 
  """
  Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
  "alphas" and the hidden states "a" of the Bi-LSTM.
  
  Arguments:
  a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
  s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
  
  Returns:
  context -- context vector, input of the next (post-attention) LSTM cell
  """
  s_prev = RepeatVector(Tx)(s_prev)
  concat = Concatenate(axis=-1)([a, s_prev])
  e = Dense(10, activation = "tanh")(concat)
  energies = Dense(1, activation = "relu")(e)
  alphas = tf.nn.softmax(energies,axis=1)
  context = Dot(axes = 1)([alphas,a])
  
  return context

In [0]:
def Resnet18_Attention(Tx, Ty, n_a, n_s, output_size,input_image_size):
  
  #define resnet
  resnet = ResNet18(input_shape = (input_image_size,1), classes = 1)
  
  X_input = Input(shape = (Tx, input_image_size,1))
  
  X = tf.keras.layers.TimeDistributed(resnet)(X_input)

  s0 = Input(shape = (n_s, ), name = 's0')
  c0 = Input(shape = (n_s, ), name = 'c0')
  s = s0
  c = c0

  #Initialize empty list of outputs
  outputs = []

  a = Bidirectional(LSTM(n_a, return_sequences=True))(X)

  for t in range(Ty):
    context = one_step_attention(a, s)
    s, _, c = LSTM(n_s, return_state = True)(context, initial_state = [s, c])
    out = Dense(output_size)(s)
    #act = tf.nn.softmax(out, axis=1)
    outputs.append(out)
  
  outputs = Average()(outputs)
  model = Model(inputs = (X_input, s0, c0), outputs = outputs)

  return model

In [0]:
Tx=10 #Number of input images
Ty=10 #Number of post-LSTM cells
n_a= n_a #Number of pre-LSTM states
n_s= n_s #Number of post-LSTM states
input_image_size=750 #Input image size
output_size=2 #output of each post-LSTM cells before applying the FINAL dense layer

In [0]:
resnet18_attention = Resnet18_Attention(Tx, Ty, n_a, n_s, output_size,input_image_size)

In [96]:
resnet18_attention.summary()
keras.utils.plot_model(resnet18_attention,show_shapes=True)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 10, 750, 1)] 0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 10, 5632)     4202368     input_4[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 16)]         0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 64)       1450240     time_distributed_1[0][0]         
____________________________________________________________________________________________

##4.2 Define callbacks

###4.2.1 Tensorboard

In [0]:
#callback: tensorboard
log_dir=r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+Attention\\" + datetime.now().strftime("%Y%m%d-%H%M%S") +"ResNet-18+Attention+10ts"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

###4.2.2 Checkpoint

In [0]:
#callback: checkpoint
filepath = r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+Attention\ResNet-18+Attention+10ts-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')

In [99]:
#clear history if necessary
tf.keras.backend.clear_session()
#strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()) ##to overwrite NCCL cross device communication as this is running in Windows
#with strategy.scope():


#model = model
#model.load_weights(r'C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+LSTM\CCCCCC-Resnet18+LSTM+10ts-42-5.2190.hdf5')
resnet18_attention.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01), 
              loss='mse', 
              metrics=['mae'])

callbacks_list = [tensorboard_callback, checkpoint]

#start training
resnet18_attention.fit(train_dataset,
          epochs=300,
          steps_per_epoch = number_of_element//batch_size,
          verbose=1,
          validation_data=val_dataset,
          validation_steps=number_of_val_element//batch_size,
          callbacks=callbacks_list
          )

Epoch 1/300
304/304 [==============================] - ETA: 0s - loss: 8053.1875 - mae: 79.3498
Epoch 00001: loss improved from inf to 8053.18750, saving model to C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+Attention\ResNet-18+Attention+10ts-01-8053.1875.hdf5
304/304 [==============================] - 140s 460ms/step - loss: 8053.1875 - mae: 79.3498 - val_loss: 5236.3335 - val_mae: 60.0042
Epoch 2/300
304/304 [==============================] - ETA: 0s - loss: 3507.6855 - mae: 46.9012
Epoch 00002: loss improved from 8053.18750 to 3507.68555, saving model to C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+Attention\ResNet-18+Attention+10ts-02-3507.6855.hdf5
304/304 [==============================] - 142s 467ms/step - loss: 3507.6855 - mae: 46.9012 - val_loss: 2298.4514 - val_mae: 36.7538
Epoch 3/300
304/304 [==============================] - ETA: 0s - loss: 1538.6628 - mae: 29.5496
Epoch 00003: loss i

KeyboardInterrupt: ignored

# 5.Model Evaluation

In [0]:
os.chdir(r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+LSTM")
model = tf.keras.models.load_model('7C-Resnet18+LSTM+10ts-48-3.9742.hdf5')

## 5.1 MAE

In [0]:
with open('C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/test/test_data', "rb") as fp:
  test_data = pickle.load(fp)

In [0]:
with open("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/test/test_labels", "rb") as fp:
  test_labels = pickle.load(fp)

In [0]:
test_data = np.asarray(test_data)

In [0]:
test_labels = np.asarray(test_labels)

In [0]:
bp_estimate=model.predict(test_data)

In [0]:
from sklearn.metrics import mean_absolute_error
sys_mae = mean_absolute_error(test_labels[:,0], bp_estimate[:,0])
dia_mae = mean_absolute_error(test_labels[:,1], bp_estimate[:,1])
print("Systolic MAE in test dataset:", round(sys_mae,1))
print("Diastolic MAE in test dataset:", round(dia_mae,1))

Systolic MAE in test dataset: 6.6
Diastolic MAE in test dataset: 4.1


##5.2 FLOPS

In [0]:
def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()
        

    with graph.as_default():
        with session.as_default():
            model = tf.keras.models.load_model(model_h5_path)

            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        
            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)
        
            return flops.total_float_ops

In [0]:
FLOPS = get_flops('CCCC-Resnet18+LSTM+10ts-72-8.5513.hdf5')
FLOPS

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


11369834

In [0]:
NFLOPS = get_flops(model)

In [0]:
NFLOPS

0